# This Notebook Simulates a Single Robot Orbiting Around a Fixed Center

## Required Imports

In [6]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np

from orbit import Fish

from interaction import Interaction
from environment import Environment
from channel import Channel
from observer import Observer
from dynamics import Dynamics

from utils import generate_distortion, generate_fish, run_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulation of Orbiting

Change parameters here, e.g. the initialization of the fish.

In [7]:
from events import Homing

run_time = 90 # in seconds
num_fish = 1
arena_size = np.array([1780, 1780, 1170])
arena_center = arena_size / 2.0
initial_spread = 2000
fish_pos = np.transpose(np.array([[1600], [1600], [50]]))
fish_vel = np.zeros((num_fish, 3))
fish_phi = np.array([7/8 * math.pi])
fish_vphi = np.zeros((num_fish, 1))
clock_freqs = 1
verbose = False

distortion = generate_distortion(type='none', magnitude=130, n=math.ceil(arena_size[0]/10)+1, show=False)
environment = Environment(
    arena_size=arena_size,
    node_pos=fish_pos,
    node_vel=fish_vel,
    node_phi=fish_phi,
    node_vphi=fish_vphi,
    distortion=distortion,
    prob_type='binary',
    conn_thres=3000,
    conn_drop=1,
    noise_magnitude=10,
    verbose=verbose
)
interaction = Interaction(environment, verbose=verbose)
channel = Channel(environment)
dynamics = Dynamics(environment, clock_freq=clock_freqs)

fish = generate_fish(
    n=num_fish, 
    channel=channel, 
    interaction=interaction,
    dynamics=dynamics,
    target_dist=130*1.75,
    lim_neighbors=[2,3],
    neighbor_weights=1.0,
    fish_max_speeds=130,
    clock_freqs=clock_freqs,
    verbose=verbose
)
channel.set_nodes(fish)

observer = Observer(fish=fish, environment=environment, channel=channel)

missing_aircraft = Homing()
for i in range(1, run_time): 
    observer.instruct(event=missing_aircraft, rel_clock=i, pos=arena_center, fish_all=True)
    
run_simulation(fish=fish, observer=observer, run_time=run_time, dark=False, white_axis=False, no_legend=True)

Please wait patiently 90 seconds. Thanks.
It's time to say bye bye!


## Prepare Data for Animation

In [8]:
# Get fish data from observer
data = np.zeros((6, run_time, num_fish+2))
data[0, :, :-2] = np.transpose(np.array(observer.x))
data[1, :, :-2] = np.transpose(np.array(observer.y))
data[2, :, :-2] = np.transpose(np.array(observer.z))
data[3, :, :-2] = np.transpose(np.array(observer.vx))
data[4, :, :-2] = np.transpose(np.array(observer.vy))
data[5, :, :-2] = np.transpose(np.array(observer.vz))

# Add center
data[0, :, -2] = 1780/2
data[1, :, -2] = 1780/2
data[2, :, -2] = 1170/2+45
data[3, :, -2] = 0
data[4, :, -2] = 0
data[5, :, -2] = -1
data[0, :, -1] = 1780/2
data[1, :, -1] = 1780/2
data[2, :, -1] = 1170/2-45
data[3, :, -1] = 0
data[4, :, -1] = 0
data[5, :, -1] = 1

## Save Data

In [9]:
np.save('orbit', data)

## Animate Simulated Data

Change the speed-up factor of the animation in the variable `speed`.

In [8]:
# Imports
import ipyvolume as ipv
from ipyvolume.moviemaker import MovieMaker
import ipywidgets as widgets
import ipywebrtc as webrtc
import matplotlib.cm as cm
import time

# Data handling
x, y, z, vx, vy, vz = data
speed = 10 # speed up animation 10 times

# Colors
v = np.sqrt(x**2 + y**2 + z**2)
v -= v.min(); v /= v.max();
colors = np.array([cm.Blues(k) for k in v])
colors[:, -2:, :] = cm.Reds(0.5) # center is red

# Figure
fig = ipv.figure()
ipv.xlim(0, 1780)
ipv.ylim(0, 1780)
ipv.zlim(0, 1170)
ipv.style.use('dark')
quiver = ipv.quiver(x, y, z, vx, vy, vz, size=10, color=colors[:,:,:3])
ipv.animation_control(quiver, interval=1000/speed)
ipv.show()

ModuleNotFoundError: No module named 'ipyvolume'

## Save Animation as html

In [ ]:
ipv.save('orbit.html')
!open 'orbit.html'